In [ ]:
from openai import OpenAI

client = OpenAI(api_key='YOUR_API_KEY')

thread_id   = ''
assistant_id= ''

스레드 생성

In [ ]:
# thread = client.beta.threads.create()
# print(thread.id)
# thread_id=thread.id

펑션 툴 생성

In [ ]:
tools = [
   {
      "type": "function",
      "function": {
            "name": "get_static_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
               "type": "object",
               "properties": {
                  "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                  },
               },
               "required": ["location"],
            },
      },
   }
]

어시스턴트 생성 (만든 툴 삽입)

In [ ]:
assistant = client.beta.assistants.create(
   instructions = "어시스턴트의 함수를 활용해 날씨를 알려주는 봇", # 지시사항
   name="Weather Bot", # 어시스턴트 이름
   tools=tools, # 사용할 툴
   model="gpt-3.5-turbo" # 사용모델
)
print(assistant.id)
assistant_id=assistant.id

스레드에 메세지 생성

In [ ]:
message = client.beta.threads.messages.create(
   thread_id = thread_id,
   role="user",
   content="USA 날씨 알려줘"
)
print(message)

스레드와 어시스턴트 실행 > 안에 생성된 메세지로 질문

In [ ]:
run = client.beta.threads.runs.create_and_poll(
   thread_id=thread_id,
   assistant_id=assistant_id,
)
print(run)

openweathermap를 사용해 날씨 가져오기

In [ ]:
import requests
import json

city = "Seoul"
apiKey = "95851ef5649594b65d0e69cbc41600c4"
lang = 'kr'
units = 'metric'
api = f"https://api.openweathermap.org/data/2.5/weather?q={city}&APPID={apiKey}&lang={lang}&units={units}"

result = requests.get(api)
result = json.loads(result.text)

temperature = result['main']['temp']

required_action으로 만든 펑션 사용

In [ ]:
tool_outputs = []

for tool in run.required_action.submit_tool_outputs.tool_calls:
   if tool.function.name == "get_static_weather":
      tool_outputs.append({
         "tool_call_id": tool.id,
         "output": str(temperature),
         # "output": "57",
      })

잘 사용됐는지 확인 required_action이 뜨면 잘 된거

In [ ]:
if run.status == 'completed':
   messages = client.beta.threads.messages.list(
      thread_id=thread_id
   )
   print(messages)
else:
   print(run.status)

스레드와 어시스턴트, 가져온 API데이터를 모두 포함하여 실행

In [ ]:
if tool_outputs:
   try:
      run = client.beta.threads.runs.submit_tool_outputs_and_poll(
         thread_id=thread_id,
         run_id=run.id,
         tool_outputs=tool_outputs
      )
      print("Tool outputs submitted successfully.")
   except Exception as e:
      print("Failed to submit tool outputs:", e)
else:
   print("No tool outputs to submit.")


그에 대한 결과값 출력

In [ ]:
if run.status == 'completed':
   messages = client.beta.threads.messages.list(
      thread_id=thread_id
   )
   print(messages)
else:
   print(run.status)